In [1]:
import pandas as pd #pandas untuk membuat dataframe(df)
from openpyxl import load_workbook #untuk berinterkasi antara python & excel file
from openpyxl.styles import *
from openpyxl.chart import *
from openpyxl.chart.shapes import GraphicalProperties
from openpyxl.chart.label import DataLabelList

In [2]:
df = pd.read_excel('supermarket_sales.xlsx')

#Penjualan Total per Gender & Product Line
df = df.pivot_table(index='Gender', 
                    columns='Product line', 
                    values='Total', 
                    aggfunc='sum').round()

df.head() #melihat data teratas

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102.0,30437.0,33171.0,18561.0,30037.0,28575.0
Male,27236.0,23868.0,22974.0,30633.0,23825.0,26548.0


In [11]:
#Membuat Automated Reporting (Table & Grafik), dan dikirim ke discords (server digital_skola_project_1)
webhook_url = 'https://discord.com/api/webhooks/1026886404732620932/kg4r9FyiwFFsDU27fzPeT7rlYPN5iMWUYzFr0_2dYC8I-3iEy51wbo5A9c929xBI57tn'
##PART 1 - LOAD DATASET
df = pd.read_excel('supermarket_sales.xlsx') #membaca data excel menggunakan pandas

#Penjualan Total per Gender & Product Line
df = df.pivot_table(index='Gender', 
                    columns='Product line', 
                    values='Total', 
                    aggfunc='sum').round()

print('Save dataframe to excel...')

output_file = df.to_excel('output_file.xlsx', 
                sheet_name='Report', 
                startrow=4) #menyimpan output ke format xlsx, jika ingin format csv bisa dibuat "pd.to_csv"

print('Save dataframe done...')



Save dataframe to excel...
Save dataframe done...


In [16]:
! pip install discord #menginstall library discord , bisa diskip jika sudah ada library tersebut tinggal tambahakan tanda '#' di depan coding

In [13]:
##PART 2 - GRAFIK
wb = load_workbook('output_file.xlsx')
wb.active = wb['Report']

In [18]:
min_column = wb.active.min_column
max_column = wb.active.max_column
min_row = wb.active.min_row
max_row = wb.active.max_row

print(min_column, max_column, min_row, max_row)

barchart = BarChart()

data = Reference(wb.active, 
                min_col=min_column+1,
                max_col=max_column,
                min_row=min_row,
                max_row=max_row
                )

categories = Reference(wb.active,
                        min_col=min_column,
                        max_col=max_column,
                        min_row=min_row+1,
                        max_row=max_row
                        )

barchart.add_data(data, titles_from_data=True)
barchart.set_categories(categories)


wb.active.add_chart(barchart, 'B12')
barchart.title = 'Sales berdasarkan Produk'
barchart.style = 2
# wb.save(output_file)


#Total dari Penjualan
import string
alphabet = list(string.ascii_uppercase)
alphabet_excel = alphabet[:max_column]
#[A,B,C,D,E,F,G]
for i in alphabet_excel:
    if i != 'A':
        wb.active[f'{i}{max_row+1}'] = f'=SUM({i}{min_row+1}:{i}{max_row})'
        wb.active[f'{i}{max_row+1}'].style = 'Currency'

wb.active[f'{alphabet_excel[0]}{max_row+1}'] = 'Total'

wb.active['A1'] = 'Sales Report'
wb.active['A2'] = '2019'
wb.active['A1'].font = Font('Arial', bold=True, size=20)
wb.active['A2'].font = Font('Arial', bold=True, size=10)

wb.save('output_file.xlsx')

#PART - 3 Send to discord
# pip3 install discord==1.7.3

def send_to_discord():
    import discord
    from discord import SyncWebhook

    webhook = SyncWebhook.from_url(webhook_url)

    with open(file='output_file.xlsx', mode='rb') as file:
        excel_file = discord.File(file)

    webhook.send('This is an automated report', 
                username='Sales Bot', 
                file=excel_file)

send_to_discord()

1 7 1 10
